In [1]:
import sys
import os
import glob
import tensorflow as tf
import numpy as np
sys.path.append("/".join(os.getcwd().split("/")[:-1]))
tf.enable_eager_execution()
#tf.InteractiveSession()

/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/az01640/P

In [2]:
from src.utils.audio import load_processed_audio

In [3]:
positives, negatives, backgrounds = load_processed_audio()

W0823 00:03:52.202810 140734803187136 deprecation_wrapper.py:119] From /Users/az01640/Projets/multrigger-word/src/utils/audio.py:40: The name tf.read_file is deprecated. Please use tf.io.read_file instead.

W0823 00:03:53.064696 140734803187136 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0823 00:03:53.076023 140734803187136 deprecation.py:323] From /Users/az01640/Projets/multrigger-word/src/utils/audio.py:42: decode_audio (from tensorflow.contrib.ffmpeg.ffmpeg_ops) is deprecated and will be removed after 2018-09-04.
Instructions for updating:
tf.contrib.ffmpeg will be removed in 2.0, the support for video and audio will continue to be provided in tens

In [60]:
def background_generator(backgrounds, targeted_size, batch_size):
    def gen():
        picked_backgrounds = random.choices(backgrounds, k=batch_size)
        yield [cut_audio_segment(background, targeted_size) for background in picked_backgrounds]
    return gen


def add_word(positives, negatives, negative_ratio):
    """
    Add words to background record
    :param words: word
    :return: new background with the word add randomly in it
    """

    def fn(background):

        if np.random.random() > negative_ratio:

            label, word_label_to_add = random.choice(list(enumerate(WORDS, 1)))
            word_to_add = random.choice(positives[word_label_to_add])

        else:
            word_to_add, label = random.choice(negatives), 0

        word_length = len(word_to_add)
        background_length = int((BACKGROUND_DURATION_MS / 1000) * FRAME_RATE)

        start, end = get_random_time_segment(word_length, background_length)

        word_to_add = tf.convert_to_tensor(np.array(word_to_add), dtype=tf.float32)

        X = tf.concat([background[:start], word_to_add, background[end:]], axis=0)

        start = int(start * SPECTROGRAM_X / background_length)
        end = int(end * SPECTROGRAM_X / background_length)

        indices = list(enumerate([0] * start + [label] * (end - start) + [0] * (SPECTROGRAM_X - end)))

        Y = tf.SparseTensor(indices=indices, values=[1] * SPECTROGRAM_X,
                            dense_shape=[SPECTROGRAM_X, len(WORDS) + 1])

        return X, tf.sparse.to_dense(Y)

    def fn_batch(backgrounds):
        return tf.map_fn(fn, backgrounds, dtype=(tf.float32, tf.int32))

    return fn_batch


def spectrogram_from_samples_batch(waveforms, labels):
    def spectrogram_from_samples(sample):
        waveform, Y = sample
        signals = tf.reshape(waveform, [1, -1])
        stfts = tf.contrib.signal.stft(signals, frame_length=FFT_FRAME_LENGTH, frame_step=FFT_FRAME_STEP, fft_length=FFT_LENGTH)
        magnitude_spectrograms = tf.abs(stfts)

        num_spectrogram_bins = magnitude_spectrograms.shape[-1].value

        linear_to_mel_weight_matrix = tf.contrib.signal.linear_to_mel_weight_matrix(NUM_MEL_BINS, num_spectrogram_bins,
                                                                                    FRAME_RATE, LOWER_EDGE_HERTZ,
                                                                                    UPPER_EDGE_HERTZ)

        mel_spectrograms = tf.tensordot(magnitude_spectrograms, linear_to_mel_weight_matrix, 1)
        log_mel_spectrograms = tf.log(mel_spectrograms + tf.keras.backend.epsilon())

        return log_mel_spectrograms, Y

    return tf.map_fn(spectrogram_from_samples, (waveforms, labels))

In [61]:
ds = (tf.data.Dataset.from_generator(background_generator(backgrounds, 
                                                          int(BACKGROUND_DURATION_MS*FRAME_RATE/1000), 32),
                                     output_types=tf.float32)
      .map(add_word(positives, negatives, .3), num_parallel_calls=N_CORES)
      .map(spectrogram_from_samples_batch, num_parallel_calls=N_CORES)
     )

In [62]:
for x in ds:
    pass
x

(<tf.Tensor: id=10583, shape=(32, 1, 311, 128), dtype=float32, numpy=
 array([[[[-1.4366564e+00, -7.7895403e-01, -4.1400132e-01, ...,
           -1.4758973e+00, -1.2643327e+00, -1.3384768e+00],
          [-2.3173638e-01, -4.1728505e-01, -2.9413161e-01, ...,
           -1.0659553e+00, -4.9174583e-01, -3.8362694e-01],
          [-3.8743931e-01,  5.4830718e-01,  8.9833796e-01, ...,
           -1.0194330e+00, -1.3470844e+00, -1.4624357e+00],
          ...,
          [-2.3242333e+00, -1.5283945e+00, -1.6930568e+00, ...,
           -2.9830725e+00, -3.3339996e+00, -3.5591652e+00],
          [-2.3304853e+00, -1.9424582e+00, -2.8560116e+00, ...,
           -2.7579789e+00, -2.8835392e+00, -3.3605485e+00],
          [-1.5824753e+00, -1.8506149e+00, -2.4174280e+00, ...,
           -2.7486868e+00, -3.0387888e+00, -3.2693532e+00]]],
 
 
        [[[-1.8292398e+00, -2.9201920e+00, -1.4381874e+00, ...,
            1.4532561e+00,  1.3839183e+00,  6.7786866e-01],
          [-3.6656349e+00, -2.2930019e+00

In [ ]:
tf.data.Dataset.from_generator(background_generator(backgrounds, int(BACKGROUND_DURATION_MS*FRAME_RATE/1000)),
                                           output_types=tf.float32)
            .map(add_word(positives, negatives, negative_ratio), num_parallel_calls=N_CORES)
            .map(spectrogram_from_samples_batch, num_parallel_calls=N_CORES)